# Extract urban areas on JNB

This code provides an example of how to calculate urban areas from gridded population data. However, it is designed explicitly to run on the GOST Jupyter notebook server, so the paths may not work.

In [1]:
import sys
import os
import rasterio

import geopandas as gpd


# Import raster helpers
sys.path.append("../../gostrocks/src")
import GOSTRocks.rasterMisc as rMisc

# Import GOST urban functions
sys.path.append("../")
import src.UrbanRaster as urban

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.1-CAPI-1.11.1 0) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
iso3 = "LKA"
out_folder = "/home/wb411133/temp/%s_URBAN" % iso3
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

urban_ext = os.path.join(out_folder, "URBAN_Extents.shp")
hd_urban_ext = os.path.join(out_folder, "HD_URBAN_Extents.shp")
pop_file = os.path.join(out_folder, "population_2020.tif")

global_bounds = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
global_pop = (
    "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"
)

inG = gpd.read_file(global_bounds)
selG = inG.loc[inG["ISO3"] == iso3]
inPop_raster = rasterio.open(global_pop)

if selG.crs != inPop_raster.crs:
    selG = selG.to_crs(inPop_raster.crs)

if not os.path.exists(pop_file):
    rMisc.clipRaster(inPop_raster, selG, pop_file)

In [3]:
curR = rasterio.open(pop_file)
urban_calculator = urban.urbanGriddedPop(curR)
urban_extents = urban_calculator.calculateUrban(
    densVal=300, totalPopThresh=5000, smooth=False, queen=False, verbose=True
)
urban_extents.to_file(urban_ext)

12:31:20	: Read in urban data
12:31:20	: Creating Shape 0
12:31:20	: Creating Shape 1000


In [4]:
urban_extents = urban_calculator.calculateUrban(
    densVal=1500, totalPopThresh=50000, smooth=True, queen=True, verbose=True
)
urban_extents.to_file(hd_urban_ext)

12:31:21	: Read in urban data
12:31:21	: Creating Shape 0


../src/UrbanRaster.py:290: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  xxGeom['geometry '] = xxGeom.buffer((popRaster.res[0] / 2))


# Reverse Geocode the urban extents

In [5]:
import reverse_geocode

In [6]:
extents = gpd.read_file(urban_ext)

extents["NAME"] = ""
extents["COUNTRY"] = ""
for idx, row in extents.iterrows():
    cur_city = reverse_geocode.search(
        [(row["geometry"].centroid.y, row["geometry"].centroid.x)]
    )
    extents.loc[idx, "COUNTRY"] = cur_city[0]["country"]
    extents.loc[idx, "NAME"] = cur_city[0]["city"]
    # print(idx)
extents.groupby("COUNTRY").sum()

,ID,Pop
COUNTRY,,
India,2289,3.211364e+04
Sri Lanka,118854,1.608292e+07


In [7]:
extents.to_file(urban_ext)

In [22]:
extents.to_file(hd_urban_ext)